In [ ]:
#importing libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
#importing the dataset
dataset = pd.read_csv('NIFTY50_all.csv', engine='python')
dataset.head(5)

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619.0,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278.0,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678.0,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913.0,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123.0,0.2741


In [ ]:
#selecting time period range
dataset = dataset[(dataset.Date >'1999-12-31')]


In [ ]:
# viewing dataset shape count of rows and columns
dataset.shape

(235192, 15)

In [ ]:
#dropping null values in dataset
dataset=dataset.dropna()
dataset.head(5)

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
866,2011-06-01,MUNDRAPORT,EQ,161.45,162.10,165.70,161.25,163.50,164.00,164.08,2574106,4.223703e+13,19171.0,1271255.0,0.4939
867,2011-06-02,MUNDRAPORT,EQ,164.00,164.00,165.15,160.15,161.15,161.25,162.17,1699298,2.755678e+13,16176.0,791462.0,0.4658
868,2011-06-03,MUNDRAPORT,EQ,161.25,161.50,162.80,159.20,161.00,161.05,161.02,1185817,1.909361e+13,14810.0,722154.0,0.6090
869,2011-06-06,MUNDRAPORT,EQ,161.05,160.50,161.10,159.05,160.00,159.85,160.09,546378,8.746905e+12,7071.0,386144.0,0.7067
870,2011-06-07,MUNDRAPORT,EQ,159.85,159.85,162.75,156.35,157.00,157.25,158.52,2193466,3.477027e+13,17865.0,1425849.0,0.6500


In [ ]:
#validation null values have been dropped
dataset.isnull().sum().sum()

0

In [ ]:
#factorizing symbol column
dataset['Symbol']=pd.factorize(dataset.Symbol)[0]

In [ ]:
#dropping of column from dataset
dataset1=dataset.drop('Series',1)
dataset1.head(2)

,Date,Symbol,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
866,2011-06-01,0,161.45,162.1,165.70,161.25,163.50,164.00,164.08,2574106,4.223703e+13,19171.0,1271255.0,0.4939
867,2011-06-02,0,164.00,164.0,165.15,160.15,161.15,161.25,162.17,1699298,2.755678e+13,16176.0,791462.0,0.4658


In [ ]:
#identifying and selecting our x and y variables from dataset
y=dataset1.iloc[:,dataset1.columns=='Volume'].values
x=dataset1.iloc[:,dataset1.columns!='Volume'].values


In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
y = scaler.fit_transform(y)
y

array([[0.00659043],
       [0.00435065],
       [0.00303598],
       ...,
       [0.02158189],
       [0.02427935],
       [0.02927778]])

In [ ]:
"""With time series data, the sequence of values is important. A simple method that we can use is to split the ordered dataset into train and test datasets. The code below calculates the index of the split point and separates the data into the training datasets with 67% of the observations that we can use to train our model, leaving the remaining 33% for testing the model."""

# split into train and test sets
train_size = int(len(y) * 0.70)
test_size = len(y) - train_size
train, test = y[0:train_size,:], y[train_size:len(y),:]
print(len(train), len(test))

84240 36104


In [ ]:
"""The function takes two arguments: the dataset, which is a NumPy array that we want to convert into a dataset, and the look_back, which is the number of previous time steps to 
use as input variables to predict the next time period — in this case defaulted to 1.

This default will create a dataset where X is the number of passengers at a given time (t) and Y is the number of passengers at the next time (t + 1).

It can be configured, and we will by constructing a differently shaped dataset in the next section.
"""

# convert an array of values into a dataset matrix
def create_dataset(y, look_back=1):
	dataX, dataY = [], []
	for i in range(len(y)-look_back-1):
		a = y[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(y[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
"""The LSTM network expects the input data (X) to be provided with a specific array structure in the form of: [samples, time steps, features].

Currently, our data is in the form: [samples, features] and we are framing the problem as 
one time step for each sample. We can transform the prepared train and test input data into the expected structure using numpy.reshape() as follows:
"""

# reshape input to be [samples, time steps, features]
import numpy as np
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
trainX.shape


(84238, 1, 1)

In [ ]:
"""We are now ready to design and fit our LSTM network for this problem.
The network has a visible layer with 1 input, a hidden layer with 4 LSTM blocks or neurons, and an output layer that makes a single value prediction. The default sigmoid activation function is used for the LSTM blocks. The network is trained for 100 epochs and a batch size of 1 is used.
"""

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)

Epoch 1/20
84238/84238 - 100s - loss: 1.7155e-04
Epoch 2/20
84238/84238 - 98s - loss: 1.5731e-04
Epoch 3/20
84238/84238 - 99s - loss: 1.5226e-04
Epoch 4/20
84238/84238 - 98s - loss: 1.5114e-04
Epoch 5/20
84238/84238 - 98s - loss: 1.5125e-04
Epoch 6/20
84238/84238 - 98s - loss: 1.5108e-04
Epoch 7/20
84238/84238 - 98s - loss: 1.5063e-04
Epoch 8/20
84238/84238 - 98s - loss: 1.5100e-04
Epoch 9/20
84238/84238 - 97s - loss: 1.5097e-04
Epoch 10/20
84238/84238 - 97s - loss: 1.5099e-04
Epoch 11/20
84238/84238 - 98s - loss: 1.5097e-04
Epoch 12/20
84238/84238 - 98s - loss: 1.5081e-04
Epoch 13/20
84238/84238 - 98s - loss: 1.5003e-04
Epoch 14/20
84238/84238 - 97s - loss: 1.5047e-04
Epoch 15/20
84238/84238 - 96s - loss: 1.5079e-04
Epoch 16/20
84238/84238 - 98s - loss: 1.5049e-04
Epoch 17/20
84238/84238 - 97s - loss: 1.5075e-04
Epoch 18/20
84238/84238 - 96s - loss: 1.5098e-04
Epoch 19/20
84238/84238 - 96s - loss: 1.5058e-04
Epoch 20/20
84238/84238 - 96s - loss: 1.5043e-04


In [ ]:
"""Once the model is fit, we can estimate the performance of the model on the train and test datasets. This will give us a point of comparison for new models.

Note that we invert the predictions before calculating error scores to ensure that performance is 
reported in the same units as the original data (Stock Volume)
"""

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# calculate root mean squared error
trainscore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.10f RMSE' % (trainscore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.10f RMSE' % (testScore))

Train Score: 0.0120098050 RMSE
Test Score: 0.0214047528 RMSE


In [ ]:
#R-Squared score for training dataset
#rsquared =r2_score(trainY, trainPredict)
#rsquared

0.591405235839739

In [ ]:
#R-Squared score for testing dataset
rsquared =r2_score(testY, testPredict)
rsquared

0.5818189214836402